# Agents: Basics

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Load the OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.agents.load_tools import get_all_tool_names
from langchain_openai import ChatOpenAI

In [3]:
# For tools/agents, use temeprature=0 to get deterministic results
llm = ChatOpenAI(temperature=0)

In [4]:
# Integrations > Tools:
# https://python.langchain.com/v0.2/docs/integrations/tools/
tools = load_tools(["llm-math"], llm=llm) # list

In [5]:
# load_tools returns a list of the tools related to the tool names we input
#len(tools) # 1
type(tools[0]) # langchain_core.tools.Tool

langchain_core.tools.Tool

In [6]:
# Get all tool names
all_tool_names = get_all_tool_names()
print(all_tool_names)

['sleep', 'wolfram-alpha', 'google-search', 'google-search-results-json', 'searx-search-results-json', 'bing-search', 'metaphor-search', 'ddg-search', 'google-lens', 'google-serper', 'google-scholar', 'google-finance', 'google-trends', 'google-jobs', 'google-serper-results-json', 'searchapi', 'searchapi-results-json', 'serpapi', 'dalle-image-generator', 'twilio', 'searx-search', 'merriam-webster', 'wikipedia', 'arxiv', 'golden-query', 'pubmed', 'human', 'awslambda', 'stackexchange', 'sceneXplain', 'graphql', 'openweathermap-api', 'dataforseo-api-search', 'dataforseo-api-search-json', 'eleven_labs_text2speech', 'google_cloud_texttospeech', 'read_file', 'reddit_search', 'news-api', 'tmdb-api', 'podcast-api', 'memorize', 'llm-math', 'open-meteo-api', 'requests', 'requests_get', 'requests_post', 'requests_patch', 'requests_put', 'requests_delete', 'terminal']


In [7]:
"llm-math" in all_tool_names

True

In [8]:
# dir: return an alphabetized list of names comprising 
# (some of) the attributes of the given object, 
# and of attributes reachable from it
# Here, we get all the AgentTypes
dir(AgentType)
# 'CHAT_CONVERSATIONAL_REACT_DESCRIPTION',
# 'CHAT_ZERO_SHOT_REACT_DESCRIPTION',
# 'CONVERSATIONAL_REACT_DESCRIPTION',
# 'OPENAI_FUNCTIONS',
# 'OPENAI_MULTI_FUNCTIONS',
# 'REACT_DOCSTORE',
# 'SELF_ASK_WITH_SEARCH',
# 'STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION',
# 'ZERO_SHOT_REACT_DESCRIPTION': no examples

['CHAT_CONVERSATIONAL_REACT_DESCRIPTION',
 'CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'CONVERSATIONAL_REACT_DESCRIPTION',
 'OPENAI_FUNCTIONS',
 'OPENAI_MULTI_FUNCTIONS',
 'REACT_DOCSTORE',
 'SELF_ASK_WITH_SEARCH',
 'STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'ZERO_SHOT_REACT_DESCRIPTION',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [9]:
# We create a Zero-shot Agent
# WARNING: initialize_agent is deprecated...
# Instead, we should use new agent constructor methods:
# create_react_agent, create_json_agent, create_structured_chat_agent, ...
agent = initialize_agent(
    tools, # LLM-Math
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True # check what is happening inside
)

c:\Users\A200239740\AppData\Local\anaconda3\envs\llms\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
agent.run("What is 123456 multiplied by 29748?") # 3672569088
# > Entering new AgentExecutor chain...
# I need to multiply 123456 by 29748.
# Action: Calculator
# Action Input: 123456 * 29748
# Observation: Answer: 3672569088
# Thought:I now know the final answer
# Final Answer: 3672569088

c:\Users\A200239740\AppData\Local\anaconda3\envs\llms\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I should use the Calculator tool to multiply these two numbers.
Action: Calculator
Action Input: 123456, 29748
Observation: Answer: 153204
Thought:I now know the final answer
Final Answer: 123456 multiplied by 29748 is 3,672,076,288

> Finished chain.


'123456 multiplied by 29748 is 3,672,076,288'

In [11]:
# Same example but with the new interface
# https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.agents
# https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html#langchain.agents.react.agent.create_react_agent
from langchain.agents.react.agent import create_react_agent
from langchain.agents import AgentExecutor

In [12]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [13]:
agent = create_react_agent(
    llm,
    tools,
    prompt
)

In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [15]:
agent_executor.invoke({"input": "What is 123456 multiplied by 29748?"})

{'input': 'What is 123456 multiplied by 29748?', 'output': '153204'}